# Load Data

In [34]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

In [35]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((p - s) + (p - c))

# ANN

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Define a custom loss function for the newsvendor problem with quantile regression
def newsvendor_quantile_loss(q, y_true, y_pred):
    error = y_true - y_pred
    return tf.keras.backend.mean(tf.maximum(q*error, (q-1)*error), axis=-1)

# Define a function to create a model
def create_model(n_hidden=1, n_neurons=30, input_shape=[3], activition='relu'): #learning_rate_input=3e-3, 
    model = Sequential()
    model.add(Dense(n_neurons, activation=activition, input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation=activition))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss=lambda y,f: newsvendor_quantile_loss(critical_ratio,y,f), optimizer=optimizer)
    return model

# Define a function to build the model
def model_builder(n_hidden=1, n_neurons=30, ): #learning_rate=3e-3
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons) #, learning_rate=learning_rate

# Create a model
model_ANN = model_builder()

# Define the parameter grid
from scipy.stats import reciprocal
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    #"learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30],
    #"activation": ['relu', 'sigmoid', 'tanh']
}

# Optimize the model using RandomizedSearchCV
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3)

# Fit the model
target_train = target_train.astype('float32')
rnd_search_cv_ANN.fit(data_train, target_train)

# Make predictions
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)
print(target_pred_ANN)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

[[1351.9865]
 [1349.6698]
 [1358.2379]
 [1357.1815]
 [1354.3877]
 [1354.3551]
 [1351.6289]
 [1349.7269]
 [1357.9375]
 [1357.5563]
 [1354.4292]
 [1356.7391]
 [1356.665 ]
 [1354.4797]
 [1362.4742]
 [1351.254 ]
 [1358.9312]
 [1361.4479]
 [1360.8158]
 [1360.306 ]
 [1352.3545]
 [1357.1376]
 [1355.3202]
 [1356.3467]
 [1348.4   ]
 [1358.347 ]
 [1355.9392]
 [1354.8037]
 [1359.0729]
 [1357.8982]
 [1356.3138]
 [1362.7747]
 [1356.4557]
 [1359.5894]
 [1357.8567]
 [1352.287 ]
 [1353.3546]
 [1363.433 ]
 [1356.388 ]
 [1357.1073]
 [1359.5151]
 [1361.474 ]
 [1356.8239]
 [1358.6982]
 [1356.9045]
 [1358.4147]
 [1361.8488]
 [1355.6141]
 [1349.2848]
 [1359.0663]
 [1350.4268]
 [1353.9802]
 [1358.0398]]


# DT

In [49]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': [10, 20, 30, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'feature_fraction': [0.8, 0.9, 1.0],
    'bagging_fraction': [0.8, 0.9, 1.0],
    'bagging_freq': [3, 4, 5, 6, 7],
}

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=critical_ratio, metric='quantile', verbose=0)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(data_train, target_train)

# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(data_test)
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM i

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\lightgbm\sklearn.py", line 1092, in fit
    super().fit(
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\lightgbm\sklear

# Costs

In [50]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * np.round(target_pred_ANN[i])
    profit_DT = (p - c) * np.round(target_pred_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if target_pred_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - np.round(target_pred_ANN[i]))
    if target_pred_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (np.round(target_pred_ANN[i]) - target_test.values[i])
    if target_pred_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - np.round(target_pred_DT[i]))
    if target_pred_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (np.round(target_pred_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT
   

# Print the overall profit
print('Overall profit for ANN: ', int(overall_profit_ANN))
print('Overstock: ', int(overstock_ANN), ' Understock: ', int(understock_ANN))
print('Overall profit for DT: ', int(overall_profit_DT))
print('Overstock: ', int(overstock_DT), ' Understock: ', int(understock_DT))

Overall profit for ANN:  53585
Overstock:  0  Understock:  493
Overall profit for DT:  74417
Overstock:  0  Understock:  102


C:\Users\lanza\AppData\Local\Temp\ipykernel_560\4077709580.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overall profit for ANN: ', int(overall_profit_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_560\4077709580.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overstock: ', int(overstock_ANN), ' Understock: ', int(understock_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_560\4077709580.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25